In [3]:
from openbb_terminal.sdk import openbb
import pandas as pd
import datetime as dt

In [5]:
start = dt.datetime(2017, 1, 1)
end = dt.datetime(2023, 12, 5)
start_date = start.strftime("%Y-%m-%d")
end_date = end.strftime("%Y-%m-%d")

port_data = openbb.stocks.screener.screener_data("new_high")
bench = openbb.stocks.load("SPY", start_date=start_date, end_date=end_date, verbose=False)
bench_return = bench['Adj Close'].pct_change().sum()

rs_values = []

for ticker in port_data["Ticker"].tolist():
    try:
        df = openbb.stocks.load(ticker, start_date=start_date, end_date=end_date, verbose=False)

        current_price = df["Adj Close"].iloc[-1]
        sma50 = df["Adj Close"].rolling(50).mean().iloc[-1]
        sma200 = df["Adj Close"].rolling(200).mean().iloc[-1]
        avg_30d_vol = df["Volume"].rolling(30).mean().iloc[-1]

        # Compute RS rank
        stock_return = df['Adj Close'].pct_change().sum()
        rs = stock_return / bench_return
        
        # check for grossly outsized move
        pct_move = df['Adj Close'].pct_change()
        threshold = pct_move.std() * 3
        outsized_move = (abs(pct_move.iloc[-1]) > threshold)

        rs_values.append((ticker, rs, current_price, sma50, sma200, avg_30d_vol, outsized_move))
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Normalize RS Rank
rs_df = pd.DataFrame(rs_values, columns=["TICKER", "RS", "LAST", "SMA50", "SMA200", "AVG_VOL_30D", "OUTSIZED_MOVE"])
rs_df['RS_PERCENTILE'] = rs_df['RS'].rank(pct=True) * 100


rs_df.dropna(how="any", axis=0, inplace=True)
rs_df

,TICKER,RS,LAST,SMA50,SMA200,AVG_VOL_30D,OUTSIZED_MOVE,RS_PERCENTILE
1,KEQU,0.318555,17.120001,17.910200,16.409050,6.700000e+02,False,13.675214
2,CASI,1.688257,5.960000,3.908200,2.779900,4.407667e+04,False,66.666667
3,DERM,0.563744,5.740000,3.527880,2.255910,1.619767e+05,False,19.658120
4,SLNO,6.278976,31.670000,25.117400,9.306345,2.499900e+05,False,99.145299
5,CERE,2.451372,35.590000,23.879000,26.625650,1.213470e+06,False,88.034188
...,...,...,...,...,...,...,...,...
112,PRSR,0.065857,10.860000,10.788120,10.434060,5.576667e+03,False,9.401709
113,KBH,2.203075,54.439999,47.931178,46.094499,1.194737e+06,False,82.051282
114,IIIN,1.021724,36.029999,31.703000,30.636669,9.693667e+04,False,37.606838
115,WSR,0.787081,11.320000,10.107307,9.417296,4.916933e+05,False,30.769231


In [6]:
rs_df[rs_df['OUTSIZED_MOVE'] == True]

,TICKER,RS,LAST,SMA50,SMA200,AVG_VOL_30D,OUTSIZED_MOVE,RS_PERCENTILE
